# Look for interesting targets to show bias with

The paper goal is to show how a clearly unrelated variable can be predicted by an X-ray when the CNN can detect the site location within the X-ray.

This notebook looks at fun and absurd variables to consider.

The questions being looked at are all multiple choice in their answers. The food questions are even ordinal. Yet the majority of research out there is using binary thresholds. This is a bit of cheat. An algorithm that randomly guesses already has an AUC of 50%. Thus papers that report an AUC of 60+% isn't saying much. It is able to detect a minor pattern beyond guessing. That could be statistical noise. To show this, we will play the same game. Thus we must find thresholds to binarize by that will tell a good story.

See TakeHomeQuestionnaire.pdf for the actual food questions

# Imports

In [1]:
import numpy as np
import os
import pandas as pd
import pickle
from sklearn.metrics import roc_auc_score

# Constants

In [2]:
OAI_PKL_PATH = '/Users/brandong.hill/code/OAI/notebooks/pkl/'
idxSlc = pd.IndexSlice

# Read in data

In [3]:
# Read in the dataframes for analysis 
allclinical_df = pd.read_pickle(open(os.path.join(OAI_PKL_PATH, 'allclinical_values.pkl'), 'rb' ))

xray_df = pd.read_pickle(open(os.path.join(OAI_PKL_PATH, 'xray_values.pkl'), 'rb' ))
xray_df = xray_df[xray_df['XRBARCD'] != '']
xray_df = xray_df[xray_df['EXAMTP'] == 'Bilateral PA Fixed Flexion Knee'] # Lets only consider xrays that we might use in the deep learning

enrollees_df = pd.read_pickle(open(os.path.join(OAI_PKL_PATH, 'enrollees_values.pkl'), 'rb' ))

# Used to map varibale descriptions to a variable name
labels_df = pd.read_pickle(open(os.path.join(OAI_PKL_PATH, 'oai_vars_labels_sources.pkl'), 'rb' ))
labels_df = labels_df.set_index('Variable')

# Merge to a single frame

In [4]:
# Map of patient IDs to barcodes, also drop 4 extraneous digits in barcode
# Result= ID: XRBARCD
barcode_site_id_df = pd.DataFrame(xray_df['XRBARCD'].reset_index('Visit', drop=True).str[4:])
print('{:,}'.format(len(barcode_site_id_df)))


# Create a list of possible targets
targets = ['INCOME', 'INCOME2', 'EDCV']

# Add in food questions
food_targets = ['FFQ' + str(num) for num in range(1,72)]
targets.extend(food_targets)

# Result= XRBARCD: ID, SITE, RACE, Visit, INCOME, INCOME2, EDCV, FFQ1, FFQ2, ....
barcode_site_id_df = barcode_site_id_df.join(enrollees_df[['SITE', 'RACE']], how='left')  # Add hospital site and patient race
barcode_site_id_df = barcode_site_id_df.join(allclinical_df.loc[idxSlc[:, 'V00'], :][targets].reset_index('Visit'), on='ID', how='left') # Add starting income and comborbidities
barcode_site_id_df = barcode_site_id_df.reset_index('ID').set_index('XRBARCD')  # Switch to index by barcode
print('{:,}'.format(len(barcode_site_id_df)))  # Sanity check, the joins shouldn't be increase the number of entries

26,522
26,522


# Check for useability

In [17]:
def remove_unused_missing_categories(series):
    # drop rows with categoricals that start with '.' (e.g. .Missing)
    missing_cats = [cat for cat in series.cat.categories if not cat[0].isdigit()]
    series = series[~series.isin(missing_cats)]
    return series.cat.remove_unused_categories().copy(deep=True) 

# See if any binarization point leads to some but not all sites being over 50%
# Returns the first binarization point where this is true
def check_useability(df, var, label):
    s = remove_unused_missing_categories(df[var])

    df = pd.concat([df['SITE'], s], axis=1, join='inner')

    # Get the percents by site (index: var values, cols: site names,
    #                           val: percent of patients with that answer at that site)
    var_by_site_df = df.groupby('SITE').value_counts(normalize=True).unstack('SITE')
    var_by_site_df = var_by_site_df.sort_index()  # Critical to ensure answers are in ordinal order  

    # Row by row, add the percentages of each row until one or more cols have over 50%
    sums = pd.Series(0, index=var_by_site_df.columns)
    for answer in var_by_site_df.index:  # Indices are question answers
        row = var_by_site_df.loc[answer]
        sums += row
        num_over_half = (sums > 0.50).sum()
        if num_over_half > 0:
            break

    if num_over_half == len(row):
        # No combination of values gives some columns over 50% and others under
        print("{}: {} doesn't work.\n Min percentage is {:.1%} with answers of {} or lower".format(var, label, sums.min(), answer))
        return None
    else:
        print("{}: {}\n With answers of {} or lower\n{}".format(var, label, answer, sums.apply(lambda x: '{:.1%}'.format(x))))
        return (sums, len(s))
    
# The prior calculation stops as soon as any site has over 50% response. This allows us to get all combinations where at least one site is above 50% and at least one below
def find_best_auc(df, var):
    s = remove_unused_missing_categories(df[var])

    df = pd.concat([df['SITE'], s], axis=1, join='inner')

    # Get the percents by site (index: var values, cols: site names,
    #                           val: percent of patients with that answer at that site)
    var_by_site_df = df.groupby('SITE').value_counts(normalize=True).unstack('SITE')
    var_by_site_df = var_by_site_df.sort_index()  # Critical to ensure answers are in ordinal order  

    # Get per site counts
    counts = df.groupby('SITE').value_counts().unstack('SITE').sum()

    sums = pd.Series(0, index=var_by_site_df.columns)
    aucs, idxs = [], []
    # Indices are how often a person eats this food
    for answer in var_by_site_df.index:
        row = var_by_site_df.loc[answer]
        sums += row
        num_over_half = (sums > 0.50).sum()
        if num_over_half == len(sums):  # All are over 50%, model would just always guess true for any site
            break
        elif num_over_half > 0:  # At least one site must have over a 50% response rate   
            # Calc the AUCs
            pos = (sums * counts).round().astype(int)
            neg = (counts - pos).astype(int)
            actual, pred = np.empty(0), np.empty(0)
            for site in sums.index:
                actual = np.append(actual, np.ones(pos[site]))
                actual = np.append(actual, np.zeros(neg[site]))
                pred = np.append(pred, np.ones(int(counts[site])) * sums[site])
            aucs.append(roc_auc_score(actual, pred))
            idxs.append(answer)
    if aucs:
        aucs = pd.Series(aucs, index=idxs)
        return (aucs.max(), aucs.idxmax(), aucs, aucs.min())
    else:
        return None

## Income

In [6]:
# Look at simplified income
var = 'INCOME2'
tmp = check_useability(barcode_site_id_df, var, labels_df.at['V00'+var, 'Label'])

INCOME2: Yearly income (>50K or <50K) (calc) doesn't work.
 Min percentage is 100.0% with answers of 2: > $50K or lower


In [7]:
# Look at more detailed income
var = 'INCOME'
tmp = check_useability(barcode_site_id_df, var, labels_df.at['V00'+var, 'Label'])

INCOME: Yearly income (calc) doesn't work.
 Min percentage is 67.4% with answers of 4: $50K to < $100K or lower


## Education

In [8]:
# Look at education
var = 'EDCV'
tmp = check_useability(barcode_site_id_df, var, labels_df.at['V00'+var, 'Label'])

EDCV: Highest grade or year of school completed (calc)
 With answers of 2: Some college or lower
SITE
A    42.1%
B    47.3%
C    20.6%
D    41.8%
E    52.2%
dtype: object


## Food

In [9]:
# New version
viable = []
all_pcts = []
cnts = {}
for var in food_targets:
    label = labels_df.at['V00'+var, 'Label'].split(':')[1]
    tmp = check_useability(barcode_site_id_df, var, label)
    if tmp is not None:
        viable.append(var)
        all_pcts.append(tmp[0])
        cnts[var] = tmp[1]
    print('\n')

# Collect all contenders into a single dataframe
contenders = pd.DataFrame(all_pcts, index=viable)

FFQ1:  eggs (include egg biscuits/Egg McMuffins (not egg substitutes)), eat how often, past 12 months
 With answers of 4: 2-3 times per month or lower
SITE
A    44.0%
B    43.5%
C    47.9%
D    50.8%
E    44.0%
dtype: object


FFQ2:  bacon/breakfast sausage (including sausage biscuit), eat how often, past 12 months
 With answers of 3: Once per month or lower
SITE
A    43.4%
B    60.7%
C    51.8%
D    52.9%
E    46.6%
dtype: object


FFQ3:  cooked cereals (e.g., oatmeal/cream of wheat/grits) eat how often, past 12 months
 With answers of 3: Once per month or lower
SITE
A    40.6%
B    49.9%
C    51.1%
D    49.4%
E    44.1%
dtype: object


FFQ4:  cold cereals (e.g., Corn Flakes/Cheerios...), eat how often, past 12 months
 With answers of 5: Once per week or lower
SITE
A    52.2%
B    48.6%
C    46.1%
D    46.0%
E    51.3%
dtype: object


FFQ5:  cereal, which eat most often doesn't work.
 Min percentage is 87.9% with answers of 3: Other cold cereal, like Corn Flakes etc or lower


FFQ6:  

In [10]:
print(list(contenders.index))
print(len(contenders))

['FFQ1', 'FFQ2', 'FFQ3', 'FFQ4', 'FFQ7', 'FFQ9', 'FFQ12', 'FFQ13', 'FFQ14', 'FFQ15', 'FFQ17', 'FFQ18', 'FFQ21', 'FFQ22', 'FFQ23', 'FFQ24', 'FFQ28', 'FFQ34', 'FFQ36', 'FFQ37', 'FFQ39', 'FFQ40', 'FFQ46', 'FFQ47', 'FFQ52', 'FFQ53', 'FFQ55', 'FFQ63', 'FFQ66', 'FFQ67', 'FFQ68', 'FFQ69', 'FFQ71']
33


In [18]:
# Go through all food questions and calculate the best possible AUCS for each question
# best_aucs - a dataframe with the best AUC for each question, and the threshold

questions = []
aucs = {}
max_aucs = []
min_aucs = []
answer = []
for var in food_targets:
    res = find_best_auc(barcode_site_id_df, var)
    if res is not None:
        questions.append(var)
        max_aucs.append(res[0])
        answer.append(res[1])
        aucs[var] = res[2]
        min_aucs.append(res[3])        
        
best_aucs = pd.DataFrame({'AUC': max_aucs, 'Freq':answer, 'min': min_aucs}, index=questions)
print(len(best_aucs))  #Sanity check, this should match the count from before
all_aucs = pd.DataFrame(aucs) # Has NA for any answer that won't work, most only have one answer with an AUC 

FFQ66
33


In [11]:
# Does anything have an AUC below 0.5 (meaning we can do well if we flip the positive case)
# This was a hack, the frequency value may not be correct for this.
print(all_aucs.T.sum(axis=1).idxmin(), all_aucs.T.sum(axis=1).min())

FFQ53 0.5220867873397199


In [12]:
# How many have more than one cut-off frequency
tmp = (~all_aucs.isna()).sum()
tmp[tmp > 1]

FFQ66    2
dtype: int64

In [13]:
# Can we use eating fried chicken skin?
best_aucs.loc['FFQ39']

AUC                      0.56089
Freq    1: Avoid eating the skin
min                      0.56089
Name: FFQ39, dtype: object

In [14]:
# What has strong AUCs?
for idx in best_aucs[best_aucs['AUC'] > 0.6].index:
    label = labels_df.at['V00'+idx, 'Label'].split(':')[1]
    if 'months' in label:
        label = label.split(',')[0] # drop the ', past 12 months' 
    print('{:.3f} {}  {}'.format(best_aucs.loc[idx]['AUC'], idx, label), best_aucs.loc[idx]['Freq'])

0.631 FFQ18   refried beans 1: Never
0.607 FFQ37   fried chicken 2: A few times per year
0.609 FFQ52   tortillas 1: Never


In [20]:
var = 'FFQ18'
tmp = check_useability(barcode_site_id_df, var, labels_df.at['V00'+var, 'Label'])

FFQ18: Block Brief 2000: refried beans, eat how often, past 12 months
 With answers of 1: Never or lower
SITE
A    59.4%
B    66.1%
C    34.5%
D    56.0%
E    60.7%
dtype: object


In [21]:
var = 'FFQ37'
tmp = check_useability(barcode_site_id_df, var, labels_df.at['V00'+var, 'Label'])

FFQ37: Block Brief 2000: fried chicken, at home or in a restaurant, eat how often, past 12 months
 With answers of 2: A few times per year or lower
SITE
A    49.8%
B    77.0%
C    65.3%
D    61.8%
E    53.0%
dtype: object


In [22]:
var = 'FFQ52'
tmp = check_useability(barcode_site_id_df, var, labels_df.at['V00'+var, 'Label'])

FFQ52: Block Brief 2000: tortillas, eat how often, past 12 months
 With answers of 1: Never or lower
SITE
A    51.4%
B    52.5%
C    27.9%
D    43.6%
E    49.1%
dtype: object
